#  OPC Corporate Back Brief Project
## Outdoor Performance Center Data Science Team 
[Krupa Jacob]
[03/10/2024]

Example query to analyze product sales:


In [1]:
SELECT p.prod_name, SUM(o.order_tot) AS total_sales
FROM products p
JOIN orders o ON p.prod_id = o.prod_id
GROUP BY p.prod_name
ORDER BY total_sales DESC;

(68 row(s) affected)

Total execution time: 00:00:00.013

prod_name,total_sales
Rail,348444.84
SB160,324004.06
Occam,319280.03
Altitude,313788.5
SB150,306540
Gnarvana,280900.44
Wreckoning,278751.8
E-Series,269617.47
Firebird,259038.38
Mojo,255649.11


Analyzing the growth of mountain bike sales over time:

In [35]:
SELECT LEFT(ord_date::text, 4) AS order_year,
       SUM(order_tot) AS total_sales
FROM orders o
JOIN products p ON o.prod_id = p.prod_id
GROUP BY LEFT(ord_date::text, 4)
ORDER BY LEFT(ord_date::text, 4);

(20 row(s) affected)

Total execution time: 00:00:00.004

order_year,total_sales
2000,728574
2001,696021.1
2002,565160.44
2003,523462.72
2004,577862.3
2005,635804.4
2006,754774.1
2007,598244.7
2008,607583.9
2009,651427.75


Comparing sales by warehouse to assess capacity:

In [13]:
SELECT w.warehouse_name, SUM(o.order_tot) AS total_sales
FROM warehouses w
JOIN orders o ON w.warehouse_id = o.warehouse_id
GROUP BY w.warehouse_name
ORDER BY total_sales DESC;

(3 row(s) affected)

Total execution time: 00:00:00.015

warehouse_name,total_sales
Sacramento,4.2647605e+06
Dallas,4.1522755e+06
Columbus,4.087617e+06


Identifying the most popular mountain bike build configurations:

In [38]:
SELECT b.build_id, p.prod_name, COUNT(*) AS total_orders
FROM builds b
JOIN productbuilds pb ON b.build_id = pb.build_id
JOIN products p ON pb.prod_id = p.prod_id
GROUP BY b.build_id, p.prod_name  -- Added GROUP BY clause
ORDER BY total_orders DESC
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.008

build_id,prod_name,total_orders
32,Occam,2
15,Sensor,1
13,X Trance,1
48,Trail Pistol,1
25,AMD,1
2,Scalpel,1
65,SB160,1
8,Rail,1
53,Stereo,1
41,SB130,1


Forecasting mountain bike sales for the next year based on historical growth rates:

In [34]:
WITH sales_growth AS (
  SELECT EXTRACT(YEAR FROM ord_date::date) AS order_year, 
         SUM(order_tot) AS total_sales,
         LAG(SUM(order_tot), 1) OVER (ORDER BY EXTRACT(YEAR FROM ord_date::date)) AS prev_year_sales
  FROM orders o
  JOIN products p ON o.prod_id = p.prod_id
  GROUP BY order_year
)
SELECT order_year + 1 AS forecast_year, 
       total_sales * (1 + (total_sales - prev_year_sales) / prev_year_sales) AS forecast_sales
FROM sales_growth
ORDER BY order_year DESC
LIMIT 1;

(1 row(s) affected)

Total execution time: 00:00:00.012

forecast_year,forecast_sales
2020,484623.12561060395
